In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv("Market_Basket_Optimisation.csv")
df=df1.iloc[:,:]
df.shape

(7500, 20)

In [2]:

transaction = []
for i in range(df.shape[0]):
    transaction.append([str(df.values[i,j]) for j in range(df.shape[1])])
# creating the numpy array of the transactions
transaction = np.array(transaction)
matrix=transaction[:,:]

matrix1 = []

# Iterate over each row in the original array
for row in matrix:
    # Filter out 'nan' values and append non-'nan' values to the new matrix
    filtered_row = [item for item in row if item != 'nan']
    matrix1.append(filtered_row)

# Convert the list of lists to a numpy array
#matrix1 = np.array(matrix1)
matrix=matrix1

In [3]:
from mlxtend.preprocessing import TransactionEncoder
temp = TransactionEncoder()
temp_df = temp.fit(matrix).transform(matrix)
new_df = pd.DataFrame(temp_df,columns=temp.columns_)
new_df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
new_df.shape

(7500, 120)

In [5]:
new_df['Count']=new_df.sum(axis=1)

l=new_df['Count'].max()


In [6]:
print("Maximum Selling item",l)

Maximum Selling item 19


In [60]:
#binary_df1
test=new_df.copy()
#test

test=test.drop(columns=['Count'])

sum_row = test.sum()
per=sum_row*100/len(new_df)
# Append the sum row to the DataFrame
test.loc['percentage'] = per


per_row = test.loc['percentage']

# Find maximum value
max_value = per_row.max()

# Find minimum value
min_value = per_row.min()

# Find average (mean) value
mean_value = per_row.mean()

# Find median value
median_value = per_row.median()

# Output the results
print(f"Maximum: {max_value}")
print(f"Minimum: {min_value}")
print(f"Average (Mean): {mean_value}")
print(f"Median: {median_value}")


Maximum: 23.826666666666668
Minimum: 0.013333333333333334
Average (Mean): 3.2597777777777774
Median: 1.5666666666666667


In [61]:
#test
# add a new col count
test['Count']=test.sum(axis=1)
T=test['Count'].max()
print("Maximum selling item in test dataset",T)
print("Before deletion the size:",test.shape)
sum_row = test.loc['percentage']
# Dropped Min_% Threshold
# 1.567 is the median and 3.259 is the average items

cols_to_drop = sum_row[sum_row < 1.567].index

print("Number of Cols dropped",len(cols_to_drop))

test = test.drop(columns=cols_to_drop)
print("After deletion the size:",test.shape)
test=test.drop(['percentage'])
test=test.astype(int)
test.shape

Maximum selling item in test dataset 391.1733333333333
Before deletion the size: (7501, 121)
Number of Cols dropped 60
After deletion the size: (7501, 61)


(7500, 61)

In [66]:
import os, psutil
min_s=.03
min_conf=.05

In [67]:
#Hybrid-Apriori
from mlxtend.frequent_patterns import apriori,association_rules
#from mlxtend.frequent_patterns import association_rules
import pandas as pd
import time
start_inter=time.time()
process1 = psutil.Process()
ap1=apriori(new_df.drop(['Count'],axis=1),min_support=min_s,use_colnames=True,max_len=l)

rules=association_rules(ap1, metric="confidence", min_threshold=min_conf)
print("Time taken",time.time()-start_inter)
print("Number of rules",len(rules))
print("Memory required by Hybrid- Apriori",process1.memory_info().rss)


Time taken 0.13101768493652344
Number of rules 36
Memory required by Hybrid- Apriori 346513408


In [68]:
#Updated DR-Apriori Algorithm
#test=test.drop(columns=['Count'])
max_test=test['Count'].max()
print("Maximum selling", max_test);
import time
start2=time.time()
process2 = psutil.Process()
ap2=apriori(test.drop(['Count'],axis=1),min_support=min_s,use_colnames=True,max_len=max_test)
rules2=association_rules(ap2, metric="confidence", min_threshold=min_conf)
end2=time.time()
print("Memory required by DR-Apriori",process2.memory_info().rss)
print("time time of Exceution:",end2-start2)
print("Number of Rules",len(rules2))

Maximum selling 19
Memory required by DR-Apriori 346599424
time time of Exceution: 0.10343265533447266
Number of Rules 36


/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [69]:
#Fp-Growth
import mlxtend
from mlxtend.frequent_patterns import fpgrowth
import time
start_fpgrowth=time.time()
Memory_Required_FpGrowth = psutil.Process()
frequent_itemsets_fpgrowth = fpgrowth(new_df.drop(['Count'],axis=1), min_support=min_s,  use_colnames=True)
from mlxtend.frequent_patterns import association_rules

# compute and print the association rules
rules_fpgrowth=association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=min_conf)
time_fpgrowth=time.time()-start_fpgrowth

print("Memory required by Fp-Growth",Memory_Required_FpGrowth.memory_info().rss)
print("Time taken by FPGrowth",time_fpgrowth)
print("Total Number of rules",len(rules_fpgrowth))

Memory required by Fp-Growth 348758016
Time taken by FPGrowth 0.8928296566009521
Total Number of rules 36
